# Introduction to Cross-Validation - Lab

## Introduction

In this lab, you'll be able to practice your cross-validation skills!


## Objectives

You will be able to:

- Compare the results with normal holdout validation
- Apply 5-fold cross validation for regression

## Let's get started

This time, let's only include the variables that were previously selected using recursive feature elimination. We included the code to preprocess below.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.datasets import load_boston

In [2]:
boston = load_boston()

In [3]:
#X = data,
#y = target
boston

{'data': array([[6.3200e-03, 1.8000e+01, 2.3100e+00, ..., 1.5300e+01, 3.9690e+02,
         4.9800e+00],
        [2.7310e-02, 0.0000e+00, 7.0700e+00, ..., 1.7800e+01, 3.9690e+02,
         9.1400e+00],
        [2.7290e-02, 0.0000e+00, 7.0700e+00, ..., 1.7800e+01, 3.9283e+02,
         4.0300e+00],
        ...,
        [6.0760e-02, 0.0000e+00, 1.1930e+01, ..., 2.1000e+01, 3.9690e+02,
         5.6400e+00],
        [1.0959e-01, 0.0000e+00, 1.1930e+01, ..., 2.1000e+01, 3.9345e+02,
         6.4800e+00],
        [4.7410e-02, 0.0000e+00, 1.1930e+01, ..., 2.1000e+01, 3.9690e+02,
         7.8800e+00]]),
 'target': array([24. , 21.6, 34.7, 33.4, 36.2, 28.7, 22.9, 27.1, 16.5, 18.9, 15. ,
        18.9, 21.7, 20.4, 18.2, 19.9, 23.1, 17.5, 20.2, 18.2, 13.6, 19.6,
        15.2, 14.5, 15.6, 13.9, 16.6, 14.8, 18.4, 21. , 12.7, 14.5, 13.2,
        13.1, 13.5, 18.9, 20. , 21. , 24.7, 30.8, 34.9, 26.6, 25.3, 24.7,
        21.2, 19.3, 20. , 16.6, 14.4, 19.4, 19.7, 20.5, 25. , 23.4, 18.9,
        35.4, 24.7, 3

In [4]:
boston_features = pd.DataFrame(boston.data,
                               columns=boston.feature_names)

In [5]:
boston_features.head().T

,0,1,2,3,4
CRIM,0.00632,0.02731,0.02729,0.03237,0.06905
ZN,18.00000,0.00000,0.00000,0.00000,0.00000
INDUS,2.31000,7.07000,7.07000,2.18000,2.18000
CHAS,0.00000,0.00000,0.00000,0.00000,0.00000
NOX,0.53800,0.46900,0.46900,0.45800,0.45800
RM,6.57500,6.42100,7.18500,6.99800,7.14700
AGE,65.20000,78.90000,61.10000,45.80000,54.20000
DIS,4.09000,4.96710,4.96710,6.06220,6.06220
RAD,1.00000,2.00000,2.00000,3.00000,3.00000
TAX,296.00000,242.00000,242.00000,222.00000,222.00000


In [6]:
b = boston_features["B"]
logdis = np.log(boston_features["DIS"])
loglstat = np.log(boston_features["LSTAT"])

# minmax scaling
boston_features["B"] = (b-min(b))/(max(b)-min(b))
boston_features["DIS"] = (logdis-min(logdis))/(max(logdis)-min(logdis))

#standardization
boston_features["LSTAT"] = (loglstat-np.mean(loglstat))/np.sqrt(np.var(loglstat))

In [7]:
X = boston_features.values
y = boston.target

## Train test split

Perform a train-test-split with a test set of 0.20.

In [8]:
from sklearn.model_selection import train_test_split

In [9]:
X_train,X_test,y_train,y_test = train_test_split(X,y, test_size=.2)

Fit the model and apply the model to the make test set predictions

In [10]:
from sklearn.linear_model import LinearRegression

In [11]:
model = LinearRegression()

In [12]:
model.fit(X_train,y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

In [13]:
model.coef_

array([-1.50224242e-01,  1.39639880e-02, -2.41683154e-02,  1.59212691e+00,
       -2.04765035e+01,  2.59218582e+00,  1.36703351e-02, -1.61012250e+01,
        3.12405736e-01, -1.29873874e-02, -8.81775117e-01,  2.32811931e+00,
       -5.29385868e+00])

Calculate the residuals and the mean squared error

In [14]:
y_pred = model.predict(X_test)

In [15]:
residuals = y_test - y_pred

In [16]:
squared_error = residuals **2

In [17]:
mean_squared_error = squared_error.mean()

In [18]:
mean_squared_error

23.373622247401567

## Cross-Validation: let's build it from scratch!

### Create a cross-validation function

Write a function k-folds that splits a dataset into k evenly sized pieces.
If the full dataset is not divisible by k, make the first few folds one larger then later ones.

We want the folds to be a list of subsets of data!

In [19]:
def kfolds(data, k):
    # Force data as pandas dataframe
    df = pd.DataFrame(data)
    num_observations = len(data)
    fold_size = num_observations//k
    leftovers = num_observations%k
    folds = []
    start_obs = 0
    for fold_n in range(1,k+1):
        if fold_n <= leftovers:
            fold = df.iloc[start_obs : start_obs+fold_size +1]
            folds.append(fold)
            start_obs += fold_size + 1
        else:
            fold = df.iloc[start_obs:start_obs+fold_size]
            folds.append(fold)
            start_obs += fold_size
    return folds
    # add 1 to fold size to account for leftovers           
    

### Apply it to the Boston Housing Data

In [20]:
# Make sure to concatenate the data again
X = pd.DataFrame(X)
y = pd.DataFrame(y)
folded_data = pd.concat([X.reset_index(drop=True), y], axis=1)

In [21]:
folded_data = kfolds(boston_features,5)
folded_data

[        CRIM    ZN  INDUS  CHAS    NOX     RM    AGE       DIS  RAD    TAX  \
 0    0.00632  18.0   2.31   0.0  0.538  6.575   65.2  0.542096  1.0  296.0   
 1    0.02731   0.0   7.07   0.0  0.469  6.421   78.9  0.623954  2.0  242.0   
 2    0.02729   0.0   7.07   0.0  0.469  7.185   61.1  0.623954  2.0  242.0   
 3    0.03237   0.0   2.18   0.0  0.458  6.998   45.8  0.707895  3.0  222.0   
 4    0.06905   0.0   2.18   0.0  0.458  7.147   54.2  0.707895  3.0  222.0   
 5    0.02985   0.0   2.18   0.0  0.458  6.430   58.7  0.707895  3.0  222.0   
 6    0.08829  12.5   7.87   0.0  0.524  6.012   66.6  0.671500  5.0  311.0   
 7    0.14455  12.5   7.87   0.0  0.524  6.172   96.1  0.700059  5.0  311.0   
 8    0.21124  12.5   7.87   0.0  0.524  5.631  100.0  0.709276  5.0  311.0   
 9    0.17004  12.5   7.87   0.0  0.524  6.004   85.9  0.743201  5.0  311.0   
 10   0.22489  12.5   7.87   0.0  0.524  6.377   94.3  0.727217  5.0  311.0   
 11   0.11747  12.5   7.87   0.0  0.524  6.009   82.

### Perform a linear regression for each fold, and calculate the training and test error

Perform linear regression on each and calculate the training and test error.

In [22]:
test_errs = []
train_errs = []
k=5

for n in range(k):
    # Split in train and test for the fold
    train = pd.concat([fold for i, fold in enumerate(folded_data) if i!=n])
    test = folded_data[n]
    # Fit a linear regression model
    model.fit(train[X.columns],train[y.columns])
    #Evaluate Train and Test Errors
    y_hat_train = model.predict(train[X.columns])
    y_hat_test = model.predict(test[X.columns])
    train_residuals = y_hat_test - train[y.columns]
    test_residuals = y_hat_test - test[y.columns]
    train_errs.append(np.mean(train_residuals.astype(float)**2))
    test_errs.append(np.mean(test_residuals.astype(float)**2))
    
print(train_errs)
print(test_errs)

KeyError: "Int64Index([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12], dtype='int64') not in index"

## Cross-Validation using Scikit-Learn

This was a bit of work! Now, let's perform 5-fold cross-validation to get the mean squared error through scikit-learn. Let's have a look at the five individual MSEs and explain what's going on.

In [24]:
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score

cv_5_results = cross_val_score(model,X,y,cv=5,scoring='neg_mean_squared_error')
cv_5_results

array([-13.0161921 , -14.62832183, -24.81432997, -55.24107773,
       -19.022338  ])

Next, calculate the mean of the MSE over the 5 cross-validations and compare and contrast with the result from the train-test-split case.

In [25]:
cv_5_results_mean = np.mean(cross_val_score(model,X,y,cv=5,scoring='neg_mean_squared_error'))
cv_5_results_mean

-25.344451926139904

##  Summary 

Congratulations! You now practiced your knowledge on k-fold crossvalidation!